In [1]:
# Importando libraries
import pandas as pd
import requests
import os
import urllib
import time
from pathlib import Path
import glob

In [2]:
def get_csv_file(path,url,name):
    s_result = urllib.request.urlopen(url)
    s_text = s_result.read().decode('utf-8')
    s_file_name = path+name+".csv"
    o_file = open(s_file_name, "w", encoding="utf8")
    o_file.write(s_text)
    o_file.close()
    
def get_file_dataframe():
    
    list_of_files = glob.glob(f'{Path.home()}/TM/var_comodity/JSONS/USDA/*.csv') # * means all if need specific format then *.csv
    #print(list_of_files)
    latest_file = max(list_of_files, key=os.path.getctime)
    filename = latest_file
    #print(filename)
    df = pd.read_csv(filename)
    return df

def delete_files_used(times):
    
    for i in range(times):
        
        list_of_files = glob.glob(f'{Path.home()}/TM/var_comodity/JSONS/USDA/*.csv') # * means all if need specific format then *.csv
        latest_file = max(list_of_files, key=os.path.getctime)
        os.remove(latest_file)
        #print('ok')
        
    return 

def download_wait(path_to_downloads):
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < 20:
        time.sleep(1)
        dl_wait = False
        for fname in os.listdir(path_to_downloads):
            if fname.endswith('.crdownload'):
                dl_wait = True
        seconds += 1
    return seconds;

In [3]:
base_path = 'C:\\Users\\guilh\\TM\\var_comodity\\JSONS\\USDA\\'

In [4]:
bov_dict = {'calf-crop':'https://api.usda-reports.penguinlabs.net/data.csv?short_desc=CATTLE+-+CALF+CROP%2C+MEASURED+IN+HEAD&year=2022&agg_level_desc=STATE',
            'beef-inv':'https://api.usda-reports.penguinlabs.net/data.csv?short_desc=CATTLE%2C+COWS%2C+BEEF+-+INVENTORY&year=2022&agg_level_desc=STATE&reference_period_desc=FIRST+OF+JAN',
            'milk_inv':'https://api.usda-reports.penguinlabs.net/data.csv?short_desc=MILK+-+PRODUCTION%2C+MEASURED+IN+%24&year=2021&agg_level_desc=STATE',
            'calves-inv':'https://api.usda-reports.penguinlabs.net/data.csv?short_desc=CATTLE%2C+INCL+CALVES+-+INVENTORY%2C+MEASURED+IN+%24&year=2022&agg_level_desc=STATE&reference_period_desc=FIRST+OF+JAN',
            'calves-inv-dollars':'https://api.usda-reports.penguinlabs.net/data.csv?short_desc=CATTLE%2C+INCL+CALVES+-+INVENTORY%2C+MEASURED+IN+%24&year=2022&agg_level_desc=STATE&reference_period_desc=FIRST+OF+JAN',
            'calves-inv-dolar-vs-head':'https://api.usda-reports.penguinlabs.net/data.csv?short_desc=CATTLE%2C+INCL+CALVES+-+INVENTORY%2C+MEASURED+IN+%24+%2F+HEAD&year=2022&agg_level_desc=STATE&reference_period_desc=FIRST+OF+JAN'}

sui_dict = {'hog-inv':'https://api.usda-reports.penguinlabs.net/data.csv?short_desc=HOGS+-+INVENTORY&year=2022&agg_level_desc=STATE&reference_period_desc=FIRST+OF+DEC',
           'hog-inv-dollar':'https://api.usda-reports.penguinlabs.net/data.csv?short_desc=HOGS+-+INVENTORY%2C+MEASURED+IN+%24&year=2021&agg_level_desc=STATE&reference_period_desc=FIRST+OF+DEC',
           'litter-rate':'https://api.usda-reports.penguinlabs.net/data.csv?short_desc=HOGS+-+LITTER+RATE%2C+MEASURED+IN+PIGS+%2F+LITTER&year=2022&agg_level_desc=STATE&reference_period_desc=MARKETING+YEAR',
           'pig-crop':'https://api.usda-reports.penguinlabs.net/data.csv?short_desc=HOGS+-+PIG+CROP%2C+MEASURED+IN+HEAD&year=2022&agg_level_desc=STATE&reference_period_desc=MARKETING+YEAR',
           'breed-inv':'https://api.usda-reports.penguinlabs.net/data.csv?short_desc=HOGS%2C+BREEDING+-+INVENTORY&year=2022&agg_level_desc=STATE&reference_period_desc=FIRST+OF+DEC',
           'mkt-inv':'https://api.usda-reports.penguinlabs.net/data.csv?short_desc=HOGS%2C+MARKET+-+INVENTORY&year=2022&agg_level_desc=STATE&reference_period_desc=FIRST+OF+DEC',
           'farrowed':'https://api.usda-reports.penguinlabs.net/data.csv?short_desc=HOGS%2C+SOWS+-+FARROWED%2C+MEASURED+IN+HEAD&year=2022&agg_level_desc=STATE&reference_period_desc=MARKETING+YEAR'}

In [5]:
dict_list = [bov_dict,sui_dict]

In [6]:
for usda_dict in dict_list:
    for keys in usda_dict.keys():
        path = base_path
        folder_name = path+keys        
    
        if not os.path.exists(folder_name):
            os.mkdir(folder_name)
    
        else:    
            pass

In [7]:
for dict_usda in dict_list:
    for name,url in dict_usda.items():
        path = base_path+name+'\\'+name
        get_csv_file(base_path,url,name)
        
        df_csv = get_file_dataframe()
        unit = str(list(df_csv['unit_desc'].unique()))
        states_list = df_csv['state_alpha'].unique()
        
        end_path = '_us_'+unit+'close_y1.json'
        
        for states in states_list:
            
            full_name = path+states+end_path.replace(" ","").replace("['","").replace('/',"").replace('$',"")    
            df_edit = df_csv.copy()
            filtro = (df_edit.state_alpha ==states)
            
            df_filter = df_edit[filtro]
            
            df_json = df_filter[['year','Value']]
            df = df_json
            if not df.empty:
            #    print(df.empty)
                df.to_json(full_name,indent =1)
            else:
                pass


In [8]:
delete_files_used(13)